In [1]:
from pymongo import MongoClient
import pandas as pd
import re
import getpass
import json
import numpy as np
from pandas.io.json import json_normalize
import math
import geopandas as gpd
from geopy.distance import distance
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [38]:
import dotenv
import json
import os
import requests
from dotenv import load_dotenv

### Primer filtro del Dataset: Startups que hayan levantado mas de 1M

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

Hacemos una query a MongoDB para filtrar el dataset:

In [3]:

df = {"$and":[{"founded_year":{"$gte":2003}},{'funding_rounds.raised_amount':{'$gte':1000000}}]}
Startups = list(db["companies"].find(df))
len(Startups)
#df = pd.DataFrame(startups)

2260

In [4]:
df = pd.DataFrame(Startups)
df.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,offices,milestones,video_embeds,screenshots,external_links,partners,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,"[{'description': None, 'address1': '135 Missis...","[{'id': 9588, 'description': 'Another Digg Exe...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[117, 150], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[],NaN,NaN,None,None
2,52cdef7c4bab8bd675297d91,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,...,"[{'description': 'Headquarters', 'address1': '...","[{'id': 15460, 'description': 'Announced hire ...","[{'embed_code': '<object width=""425"" height=""3...",[],[],[],NaN,NaN,None,None
3,52cdef7c4bab8bd675297d96,Gizmoz,gizmoz,http://www.crunchbase.com/company/gizmoz,http://gizmoz.com,,,None,web,NaN,...,"[{'description': None, 'address1': None, 'addr...",[],[],[],[],[],NaN,NaN,None,None
4,52cdef7c4bab8bd675297d97,Scribd,scribd,http://www.crunchbase.com/company/scribd,http://scribd.com,http://blog.scribd.com/,http://blog.scribd.com/feed/,scribd,news,50.0,...,"[{'description': 'HQ', 'address1': '539 Bryant...","[{'id': 4960, 'description': 'Scribd launches ...","[{'embed_code': '<script src=""http://i.cdn.tur...","[{'available_sizes': [[[150, 72], 'assets/imag...",[{'external_url': 'http://www.online-tech-tips...,[],NaN,NaN,None,None


Nos quedmaos con las columnas que queremos:

In [5]:
offices = df[["name","offices"]]
offices.head()

,name,offices
0,Wetpaint,"[{'description': '', 'address1': '710 - 2nd Av..."
1,Digg,"[{'description': None, 'address1': '135 Missis..."
2,Geni,"[{'description': 'Headquarters', 'address1': '..."
3,Gizmoz,"[{'description': None, 'address1': None, 'addr..."
4,Scribd,"[{'description': 'HQ', 'address1': '539 Bryant..."


Hacemos 'Explode' en la columna 'offices' y transformamos cada elemento de una lista en una fila, replicando los valores del índice:

In [6]:
clean1 = offices.explode("offices")
clean2 = clean1[["offices"]].apply(lambda r: r.offices ,result_type="expand", axis=1)

officePoint = pd.concat([clean1,clean2],axis=1)
officePoint.head()

,name,offices,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Wetpaint,"{'description': '', 'address1': '710 - 2nd Ave...",,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
0,Wetpaint,"{'description': '', 'address1': '270 Lafayette...",,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
1,Digg,"{'description': None, 'address1': '135 Mississ...",None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
2,Geni,"{'description': 'Headquarters', 'address1': '9...",Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
3,Gizmoz,"{'description': None, 'address1': None, 'addre...",None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472


Eliminamos la columna office ya que lo tenemos desglosado en las otras columnas:

In [7]:
officePoint = officePoint.drop(columns=["offices"])
officePoint.head()

,name,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Wetpaint,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
0,Wetpaint,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
1,Digg,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
2,Geni,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
3,Gizmoz,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472


Eliminamos aquellas filas que no tengan coordenadas ya que no nos harian falta:

In [8]:
officePoint.isnull().sum()[officePoint.isnull().sum() > 0]
NonNaN = {'latitude':'UNKNOWN',
          'longitude':'UNKNOWN'
          }
officePoint.fillna(value=NonNaN, inplace = True)

In [9]:
point = officePoint[officePoint['latitude'] != "UNKNOWN"]
point.head()

,name,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Wetpaint,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.6031,-122.333
0,Wetpaint,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.7237,-73.9964
1,Digg,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.7647,-122.395
2,Geni,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.0904,-118.393
3,Gizmoz,None,None,None,None,Menlo Park,CA,USA,37.4841,-122.169


Creamos el 'point' con la siguiente función:

In [29]:
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
        

point["location"] = point[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
startupsRicas = point[["latitude","longitude"]]
startupsRicas

<ipython-input-29-18c939a5b960>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  point["location"] = point[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)


,latitude,longitude
0,47.6031,-122.333
0,40.7237,-73.9964
1,37.7647,-122.395
2,34.0904,-118.393
3,37.4841,-122.169
...,...,...
2253,37.3778,-121.974
2254,49.2825,-123.109
2255,37.5252,-121.918
2257,37.4846,-122.182


In [35]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

In [44]:
def requestMap(lat, long, tipo):
    coor = lat +','+ long
    token = os.getenv("TOKEN_GOOGLE")
    url1 = url
    parametros = {"location":coor, "radius": 20000, "type":tipo, "key":token}
    res=requests.get(url1, parametros=parametros)
    if res.status_code != 200:
        print(res.text)
        raise ValueError("Bad Response")
    return res.json()


In [45]:
requestMap(40,-3,"airport")

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [11]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [12]:
start_lat = 40.408561
start_lon = -3.6917665
heat_m = folium.Map(location=[start_lat, start_lon],tiles='cartodbpositron', zoom_start=2)

In [ ]:
uni_lon, uni_lat = -3.6917665, 40.408561
Marker([uni_lat, uni_lon], icon=folium.Icon(color='red')).add_to(heat_m)
heat_m

# STARBUCKS

In [14]:
starbucks = pd.read_csv("/Users/santiagobermejo/Desktop/IRONHACK/Proyecto-MongoDB/output/directory.csv")

In [16]:
starbucks.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


In [17]:
CoorStarbucks = df[['Longitude', 'Latitude' ]]

In [19]:
CoorStarbucks.head()

,Longitude,Latitude
0,1.53,42.51
1,55.47,25.42
2,55.47,25.39
3,54.38,24.48
4,54.54,24.51


# AEREOPUERTOS

In [20]:
airports = pd.read_csv("/Users/santiagobermejo/Desktop/IRONHACK/Proyecto-MongoDB/output/airports.csv")

In [21]:
airports

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55788,317861,ZYYK,medium_airport,Yingkou Lanqi Airport,40.542524,122.358600,0.0,AS,CN,CN-21,Yingkou,yes,ZYYK,YKH,NaN,NaN,https://en.wikipedia.org/wiki/Yingkou_Lanqi_Ai...,NaN
55789,32753,ZYYY,medium_airport,Shenyang Dongta Airport,41.784401,123.496002,NaN,AS,CN,CN-21,Shenyang,no,ZYYY,NaN,NaN,NaN,NaN,NaN
55790,46378,ZZ-0001,heliport,Sealand Helipad,51.894444,1.482500,40.0,EU,GB,GB-ENG,Sealand,no,NaN,NaN,NaN,http://www.sealandgov.org/,https://en.wikipedia.org/wiki/Principality_of_...,Roughs Tower Helipad
55791,307326,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584278,47.296389,11.0,AF,TF,TF-U-A,Grande Glorieuse,no,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
airports["type"].value_counts()

small_airport     34236
heliport          11449
medium_airport     4540
closed             3904
seaplane_base      1021
large_airport       618
balloonport          25
Name: type, dtype: int64

In [27]:
nanAriports = airports[airports['type'] != "heliport"] | airports[airports['type'] != "closed"]

TypeError: unsupported operand type(s) for |: 'float' and 'bool'